In [1]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.linear_model import LogisticRegression, LinearRegression, Ridge
from sklearn.pipeline import make_pipeline
import pandas as pd
import numpy as np


In [2]:
data_6m = pd.read_csv('./datas_after_tokenizer/6m/6m_after_prepro.csv')
data_7m = pd.read_csv('./datas_after_tokenizer/7m/7m_after_prepro.csv')
data_8m = pd.read_csv('./datas_after_tokenizer/8m/8m_after_prepro.csv')
data_9m = pd.read_csv('./datas_after_tokenizer/9m/9m_after_prepro.csv')

In [3]:
df_6m = data_6m.sample(frac=1).reset_index(drop=True)
df_7m = data_7m.sample(frac=1).reset_index(drop=True)
df_8m = data_8m.sample(frac=1).reset_index(drop=True)
df_9m = data_9m.sample(frac=1).reset_index(drop=True)
print(df_6m.iloc[0:10,2])

0    0
1    0
2    3
3    3
4    3
5    0
6    3
7    0
8    3
9    0
Name: 1, dtype: int64


In [4]:
X_6m = df_6m.iloc[:, 1].values
y_6m = df_6m.iloc[:, 2].values
X_7m = df_7m.iloc[:, 1].values
y_7m = df_7m.iloc[:, 2].values
X_8m = df_8m.iloc[:, 1].values
y_8m = df_8m.iloc[:, 2].values
X_9m = df_9m.iloc[:, 1].values
y_9m = df_9m.iloc[:, 2].values

In [5]:
X_train_text_6m, X_test_text_6m, y_train_6m, y_test_6m = train_test_split(X_6m, y_6m, test_size=0.2)
X_train_text_7m, X_test_text_7m, y_train_7m, y_test_7m = train_test_split(X_7m, y_7m, test_size=0.2)
X_train_text_8m, X_test_text_8m, y_train_8m, y_test_8m = train_test_split(X_8m, y_8m, test_size=0.2)
X_train_text_9m, X_test_text_9m, y_train_9m, y_test_9m = train_test_split(X_9m, y_9m, test_size=0.2)

# 아니다 여기서는 합치는게 맞는 것 같다. numpy 

https://stackoverflow.com/questions/21887754/numpy-concatenate-two-arrays-vertically

참고해서. 왜냐하면 tf-idf값을 추출하려면 훈련 단계별로 접근하는데 여기서 4번 훈련하면 4단계로 나뉘어져 있어서 문서 전체의 값을 뽑기 힘들어서. 만약 이 방법이 있으면 합치지 않아도 됌.

http://kavita-ganesan.com/extracting-keywords-from-text-tfidf/#.W7ycdmgzZPY

https://stackoverflow.com/questions/25217510/how-to-see-top-n-entries-of-term-document-matrix-after-tfidf-in-scikit-learn

이 글이 좋음. 이걸로 해보자.


In [6]:
print(len(X_train_text_6m))
print(X_train_text_6m[0])
print(y_train_6m[:10])

161144
한국 주한 미군 평택 기지 이전 미 동맹 발전 기대 오류 우회 위한 함수 추가 서울 뉴시스 사진 공 동취 재단 주한 미군 일 경기도 평택 캠프 험프리스 주한 미군 사령부 개관식 공식 업무 사진 사령부 전경 서울 뉴시스 정윤아 기자 자유 국당 일 주한 미군 사령부 평택 기지 이전 관련 해 평택 기지 기반 한 동맹 더욱 굳건 발전 기대 한다고 윤영석 수석 대변인 이날 구두 논평 통해 미동 맹 단순 군사 동맹 피 혈맹 자유민주주의 시장 경제 가치 공유 동맹 같이 말 했 윤 대변인 최근 북 핵 협상 과정 한미 연합 군사 훈련 중단 연기 됐 주한 미군 철수 거론 전시 작전 전환 논의 속도 설명 했 그 상황 동북아 미래 고려 해 한국 미국 함께 건설 평택 기지 의미 남다르 뉴시스 빅 데이터 주가 시세표 바로 뉴시스 페이스북 트위터 저작권자 공감 언론 뉴시스 통신사 무단 전재 배포 금지 
[0 1 2 2 0 0 2 0 0 1]


In [7]:
pipe = make_pipeline(TfidfVectorizer(min_df=5), LogisticRegression())
param_grid = {
    'logisticregression__C' : [0.001, 0.01, 0.1, 1, 10]
}

grid = GridSearchCV(pipe, param_grid, cv=5)
#print(y_train.shape)
grid.fit(X_train_text_6m, y_train_6m)
grid.fit(X_train_text_7m, y_train_7m)
grid.fit(X_train_text_8m, y_train_8m)
grid.fit(X_train_text_9m, y_train_9m)
print("최상의 교차 검증 점수 : %.2f" %(grid.best_score_))

최상의 교차 검증 점수 : 0.97


In [8]:
print(" 테스트 점수 : %.2f" %(grid.score(X_test_text_6m, y_test_6m)))
print(" 테스트 점수 : %.2f" %(grid.score(X_test_text_7m, y_test_7m)))
print(" 테스트 점수 : %.2f" %(grid.score(X_test_text_8m, y_test_8m)))
print(" 테스트 점수 : %.2f" %(grid.score(X_test_text_9m, y_test_9m)))

 테스트 점수 : 0.98
 테스트 점수 : 0.98
 테스트 점수 : 0.98
 테스트 점수 : 0.97


In [9]:
print('done')

done


In [10]:
vector = grid.best_estimator_.named_steps["tfidfvectorizer"]

In [12]:
indices = np.argsort(vector.idf_)[::-1]
features = vector.get_feature_names()
top_n = 2
top_features = [features[i] for i in indices[:top_n]]
print(top_features)

['오스터가드', '가본적']


In [13]:


'''
X_train_6m = vector.transform(X_train_text_6m)
X_train_7m = vector.transform(X_train_text_7m)
X_train_8m = vector.transform(X_train_text_8m)
X_train_9m = vector.transform(X_train_text_9m)
#특성별로 가장 큰 값을 찾는다.
max_value = X_train.max(axis=0).toarray().ravel()
sorted_by_tfidf = max_value.argsort()
'''
#특성 이름을 구한다.
X_train_6m = vector.transform(X_train_text_6m)
feature_names = np.array(vector.get_feature_names())
max_value = X_train_6m.max(axis=0).toarray().ravel()
sorted_by_tfidf = max_value.argsort()

print("6월 tfidf가 가장 낮은 특성 20 개 : \n", feature_names[sorted_by_tfidf[:20]])
print("6월 tfidf가 가장 높은 특성 20 개 : \n ", feature_names[sorted_by_tfidf[-20:]])

sorted_by_idf = np.argsort(vector.idf_)
print("6월 idf가 가장 낮은 특성 100개 \n", feature_names[sorted_by_idf[:100]])


6월 tfidf가 가장 낮은 특성 20 개 : 
 ['여영국' '엘리오' '엘리야' '엘리아스' '엘리시움' '팔람' '엘룬' '엘로라' '엘렌쇼' '엘렉트라' '팔렛' '엘란'
 '엔포' '엔키' '엔조' '엔젤리' '엔아이' '꿀통' '꿈결' '꿈꾼']
6월 tfidf가 가장 높은 특성 20 개 : 
  ['일품진로' '한서희' '탁현민' '김현중' '젤라' '김광석' '쉐어링' '코딩' '수경' '바오' '머크' '수박' '프로젝터'
 '보이차' '타이어' '손지현' '복권' '틀니' '육우' '티맥스']
6월 idf가 가장 낮은 특성 100개 
 ['기자' '금지' '배포' '무단' '위한' '추가' '오류' '우회' '함수' '전재' '뉴스' '서울' '사진' '이미지'
 '오후' '원본' '경제' '통해' '저작권자' '지난' '한다' '위해' '바로' '채널' '대한' '함께' '네이버' '한국'
 '이날' '제보' '페이스북' '이번' '된다' '진행' '구독' '열린' '대표' '제공' '방송' '시간' '가능' '관련'
 '시장' '시작' '기사' '이후' '예정' '공개' '대해' '이상' '사람' '우리' '정부' '설명' '모습' '확인'
 '특히' '한편' '최근' '미국' '참석' '스타' '관계자' '연합뉴스' '문제' '정보' '이야기' '상황' '지방' '계획'
 '선거' '오늘' '대통령' '필요' '해야' '모두' '자신' '현재' '발표' '영상' '언론' '기업' '세계' '지역'
 '코리아' '가장' '따라' '올해' '경우' '결과' '국내' '다양' '가운데' '공식' '지난해' '직접' '사업' '지원'
 '적극' '주요']


In [14]:
print("6월 tfidf가 가장 높은 특성 1000 개 : \n ", feature_names[sorted_by_tfidf[-1000:]])

6월 tfidf가 가장 높은 특성 1000 개 : 
  ['정려원' '서효림' '열전' '카풀' '디바이스' '이디야' '포르쉐' '개헌' '김지나' '오징어' '예약' '듀플렉스'
 '버거킹' '아마존' '에브라' '김가연' '아로나' '김창옥' '한샘' '전미라' '소유진' '복지' '커큐민' '패션' '광해'
 '카이저' '수출' '로제' '잇몸' '타짜' '월렛' '프니엘' '스프링스' '윤다영' '모기' '할머니' '윤소이' '이준기'
 '성종' '클라이밍' '옵션' '워치' '장어' '롤스로이스' '도마뱀' '신약' '낙뢰' '항만' '한화' '설계사' '연습생'
 '송주명' '황교안' '가왕' '사크' '이진아' '트리' '유영철' '동춘' '영준' '샌드위치' '윤상현' '건조기' '재이'
 '직원' '전력량계' '한우' '임현주' '어벤져스' '김준현' '매장' '쌀보리' '윤서인' '닭발' '선글라스' '논문'
 '테더' '김성은' '위니' '시반' '유승옥' '매트리스' '제이지' '김강우' '업비트' '빌라' '페르마' '강기영' '시리'
 '제이알' '르네' '금보' '홍콩' '라노스' '빨대' '이본' '아워홈' '소프트웨어' '도시바' '리사' '하산' '아세안'
 '윤리' '독전' '꽈배기' '은태' '엘리' '미얀마' '월향' '노치' '다이슨' '홍윤화' '정은채' '전북은행' '증여'
 '슐츠' '현영' '송은이' '맥거핀' '신승호' '소켓' '이윤지' '이승엽' '어비스' '정화' '렉서스' '권영찬' '효민'
 '쿠딩' '카리오' '길해연' '도파민' '구례' '채연' '김종국' '가비' '모르타르' '음료' '레옹' '리갈' '이케아'
 '김기덕' '옥상달빛' '귀농' '캐처' '김일중' '니지' '화물창' '림프관' '아바타' '대만' '함소원' '문지애' '제니'
 '지진' '소진' '이수근' '력사' '랜드' '스카' '페루' '뉴본' '빕스' '서장훈' '정경호' '러빈' '보훈' '비투비'
 '태양광' '배넌'

In [15]:

#특성 이름을 구한다.
X_train_7m = vector.transform(X_train_text_7m)
feature_names = np.array(vector.get_feature_names())
max_value = X_train_7m.max(axis=0).toarray().ravel()
sorted_by_tfidf = max_value.argsort()

print("7월 tfidf가 가장 낮은 특성 20 개 : \n", feature_names[sorted_by_tfidf[:20]])
print("7월 tfidf가 가장 높은 특성 20 개 : \n ", feature_names[sorted_by_tfidf[-20:]])

sorted_by_idf = np.argsort(vector.idf_)
print("7월 idf가 가장 낮은 특성 100개 \n", feature_names[sorted_by_idf[:100]])


7월 tfidf가 가장 낮은 특성 20 개 : 
 ['콜슨' '심영순' '정몽혁' '이엘라' '오트만' '페터슨' '심은정' '오트레드' '오토봇' '자왕' '최완' '자와바랏'
 '자옥' '경종민' '정몽일' '차리' '차릴까' '떠내려간' '키노피오' '최예근']
7월 tfidf가 가장 높은 특성 20 개 : 
  ['머크' '골판지' '육회' '수박' '타이어' '정미홍' '진세연' '가구' '프로젝터' '김옥빈' '보이차' '저글링'
 '손지현' '암웨이' '복권' '루돌프' '육우' '틀니' '아보카도' '티맥스']
7월 idf가 가장 낮은 특성 100개 
 ['기자' '금지' '배포' '무단' '위한' '추가' '오류' '우회' '함수' '전재' '뉴스' '서울' '사진' '이미지'
 '오후' '원본' '경제' '통해' '저작권자' '지난' '한다' '위해' '바로' '채널' '대한' '함께' '네이버' '한국'
 '이날' '제보' '페이스북' '이번' '된다' '진행' '구독' '열린' '대표' '제공' '방송' '시간' '가능' '관련'
 '시장' '시작' '기사' '이후' '예정' '공개' '대해' '이상' '사람' '우리' '정부' '설명' '모습' '확인'
 '특히' '한편' '최근' '미국' '참석' '스타' '관계자' '연합뉴스' '문제' '정보' '이야기' '상황' '지방' '계획'
 '선거' '오늘' '대통령' '필요' '해야' '모두' '자신' '현재' '발표' '영상' '언론' '기업' '세계' '지역'
 '코리아' '가장' '따라' '올해' '경우' '결과' '국내' '다양' '가운데' '공식' '지난해' '직접' '사업' '지원'
 '적극' '주요']


In [16]:
print("7월 tfidf가 가장 높은 특성 1000 개 : \n ", feature_names[sorted_by_tfidf[-1000:]])

7월 tfidf가 가장 높은 특성 1000 개 : 
  ['황인선' '프린팅' '레페' '폭스콘' '조민수' '손담비' '플란트' '허준석' '쏘울' '올림푸스' '류이치' '서태화'
 '애니팡' '사해' '예절' '양수경' '브라이언트' '봉태규' '미분양' '멜론' '이본' '윤후' '발사체' '주택' '퀄컴'
 '노홍철' '윤두준' '게이밍' '배당' '이규빈' '최은주' '출생' '베리' '김제동' '리넨' '왕석현' '특허' '박시후'
 '동천' '양지원' '건조기' '닉쿤' '어묵' '고속버스' '방광' '빅토리' '네이마르' '생식기' '코웰' '광고' '박보검'
 '남규리' '손병호' '진폐' '향군' '종현' '윈도' '성령' '임원희' '한승연' '박순천' '조장혁' '액티브' '이재명'
 '김승환' '박정수' '콜라겐' '김부각' '망고' '수현' '메로나' '안랩' '김도연' '다이코' '박수홍' '봄버맨' '본드'
 '공룡' '오설록' '의원' '배변' '박화영' '김동성' '화폐' '안선영' '순천' '다시마' '레오나르도' '치매' '조난자'
 '브리오니' '손나은' '메이커스' '갤럭시' '의령' '이송희일' '이준호' '대현' '노총' '연가' '육계' '재산세'
 '최희' '화웨이' '아궁' '박기영' '송해' '가영' '고구마' '심진화' '환율' '윤미래' '블루베리' '안승균' '멘토'
 '곶감' '휴켐스' '석탄' '근로' '스페이드' '효린' '강예빈' '도자기' '오지오' '김명수' '토사' '예진' '정인'
 '데이즈' '박미선' '하지원' '주스' '미세먼지' '산체스' '관광' '프라이어' '오연아' '댓글' '그루먼' '배임죄'
 '아이언' '신현수' '휴게소' '베컴' '김상중' '마돈나' '이성미' '최민환' '산다라박' '클로렐라' '기촉' '박정민'
 '정승환' '설계사' '하겐다즈' '두피' '이태환' '김준현' '리치먼드' '화장실' '강남' '자몽' '골렘' '파운드리'
 '행성' '아크' '황지은

In [17]:

#특성 이름을 구한다.
X_train_8m = vector.transform(X_train_text_8m)
feature_names = np.array(vector.get_feature_names())
max_value = X_train_8m.max(axis=0).toarray().ravel()
sorted_by_tfidf = max_value.argsort()

print("8월 tfidf가 가장 낮은 특성 20 개 : \n", feature_names[sorted_by_tfidf[:20]])
print("8월 tfidf가 가장 높은 특성 20 개 : \n ", feature_names[sorted_by_tfidf[-20:]])

sorted_by_idf = np.argsort(vector.idf_)
print("8월 idf가 가장 낮은 특성 100개 \n", feature_names[sorted_by_idf[:100]])


8월 tfidf가 가장 낮은 특성 20 개 : 
 ['마푸토' '이현두' '이헌우' '이해주' '이한진' '배치기' '이학상' '이하응' '이하윤' '배효원' '배수찬' '황인숙'
 '백련사' '이택휘' '백봉기' '호두과자' '백수련' '백승엽' '건포' '백양대로']
8월 tfidf가 가장 높은 특성 20 개 : 
  ['프로젝터' '전차' '김옥빈' '보이차' '결핵' '와인' '저글링' '타이어' '손지현' '스팀' '지부' '복권' '게놈'
 '루돌프' '모나미' '로이킴' '티맥스' '틀니' '아보카도' '육우']
8월 idf가 가장 낮은 특성 100개 
 ['기자' '금지' '배포' '무단' '위한' '추가' '오류' '우회' '함수' '전재' '뉴스' '서울' '사진' '이미지'
 '오후' '원본' '경제' '통해' '저작권자' '지난' '한다' '위해' '바로' '채널' '대한' '함께' '네이버' '한국'
 '이날' '제보' '페이스북' '이번' '된다' '진행' '구독' '열린' '대표' '제공' '방송' '시간' '가능' '관련'
 '시장' '시작' '기사' '이후' '예정' '공개' '대해' '이상' '사람' '우리' '정부' '설명' '모습' '확인'
 '특히' '한편' '최근' '미국' '참석' '스타' '관계자' '연합뉴스' '문제' '정보' '이야기' '상황' '지방' '계획'
 '선거' '오늘' '대통령' '필요' '해야' '모두' '자신' '현재' '발표' '영상' '언론' '기업' '세계' '지역'
 '코리아' '가장' '따라' '올해' '경우' '결과' '국내' '다양' '가운데' '공식' '지난해' '직접' '사업' '지원'
 '적극' '주요']


In [18]:
print("8월 tfidf가 가장 높은 특성 1000 개 : \n ", feature_names[sorted_by_tfidf[-1000:]])

8월 tfidf가 가장 높은 특성 1000 개 : 
  ['이윤미' '우울증' '영풍' '하자드' '원자력' '혈압' '욕실' '올레드' '한라' '새만금' '산양' '이성민' '김나영'
 '큐리' '신다은' '토큰' '구미' '지현' '진서연' '가민' '매그넘' '강소라' '검색어' '캠핑' '반도체' '개리'
 '최민환' '루다' '전성희' '항만' '양희은' '김명수' '물회' '유학' '크루즈' '김미애' '적립금' '신정환' '자동문'
 '고두심' '낸시랭' '인벤' '막걸리' '넷플릭스' '강경헌' '태진아' '주유소' '하지원' '치킨' '김재욱' '황신혜'
 '도서관' '정겨운' '밀크티' '찰리' '권혁수' '루보' '박준형' '하일' '보쉬' '유민' '해수욕장' '양미라' '양귀비'
 '이윤지' '회전율' '김원우' '수돗물' '정려원' '아키타견' '미역' '이국주' '전준영' '억분' '지진' '서풍' '판사'
 '제조업' '브루' '멜로디데이' '니지' '알바' '홍진호' '코코넛' '임형준' '이기광' '페인트' '장영란' '알비'
 '낙지' '이찬오' '미분양' '오윤아' '퓨전' '피처' '구글' '신봉선' '카카오' '윤활유' '김우리' '선예' '하희라'
 '강래연' '김기방' '에이핑크' '김향기' '이상민' '송지은' '카우스' '이앙기' '목동' '피파' '배윤정' '권상우'
 '화회' '김현우' '유나' '이승기' '젠지' '신세계' '백종원' '조각' '농지' '박원숙' '페트병' '핵융합' '써니'
 '이기우' '최희준' '공유' '박규리' '아이언' '유빈' '휘슬러' '엘제이' '강석' '미스코리아' '민주' '장희영'
 '태영' '우진' '이시언' '골재' '효민' '신승훈' '이규복' '금리' '산림' '국적' '몽고' '함은정' '벼락'
 '박호산' '악력' '체크' '박명수' '프레데터' '설계사' '이광재' '냉면' '유준상' '이하늘' '전표' '녹색당' '적조'
 '두피' '인화' '공천' '어촌'

In [19]:

#특성 이름을 구한다.
X_train_9m = vector.transform(X_train_text_9m)
feature_names = np.array(vector.get_feature_names())
max_value = X_train_9m.max(axis=0).toarray().ravel()
sorted_by_tfidf = max_value.argsort()

print("9월 tfidf가 가장 낮은 특성 20 개 : \n", feature_names[sorted_by_tfidf[:20]])
print("9월 tfidf가 가장 높은 특성 20 개 : \n ", feature_names[sorted_by_tfidf[-20:]])

sorted_by_idf = np.argsort(vector.idf_)
print("9월 idf가 가장 낮은 특성 100개 \n", feature_names[sorted_by_idf[:100]])


9월 tfidf가 가장 낮은 특성 20 개 : 
 ['주변광' '율리시스' '금천역' '작구' '관악역' '도봉역' '발작성' '가톨릭의료원' '정낙균' '조석구' '김춘추'
 '김동집' '엄기성' '마려워' '방광경' '서호경' '김송철' '유문부' '윤동섭' '조기진']
9월 tfidf가 가장 높은 특성 20 개 : 
  ['조현영' '와인' '저글링' '신협' '베리' '타이어' '손지현' '스팀' '암웨이' '복권' '지부' '게놈' '제대혈'
 '방귀' '모나미' '로이킴' '아이폰' '틀니' '육우' '티맥스']
9월 idf가 가장 낮은 특성 100개 
 ['기자' '금지' '배포' '무단' '위한' '추가' '오류' '우회' '함수' '전재' '뉴스' '서울' '사진' '이미지'
 '오후' '원본' '경제' '통해' '저작권자' '지난' '한다' '위해' '바로' '채널' '대한' '함께' '네이버' '한국'
 '이날' '제보' '페이스북' '이번' '된다' '진행' '구독' '열린' '대표' '제공' '방송' '시간' '가능' '관련'
 '시장' '시작' '기사' '이후' '예정' '공개' '대해' '이상' '사람' '우리' '정부' '설명' '모습' '확인'
 '특히' '한편' '최근' '미국' '참석' '스타' '관계자' '연합뉴스' '문제' '정보' '이야기' '상황' '지방' '계획'
 '선거' '오늘' '대통령' '필요' '해야' '모두' '자신' '현재' '발표' '영상' '언론' '기업' '세계' '지역'
 '코리아' '가장' '따라' '올해' '경우' '결과' '국내' '다양' '가운데' '공식' '지난해' '직접' '사업' '지원'
 '적극' '주요']


In [20]:
print("9월 tfidf가 가장 높은 특성 1000 개 : \n ", feature_names[sorted_by_tfidf[-1000:]])

9월 tfidf가 가장 높은 특성 1000 개 : 
  ['갈치' '서풍' '복무' '홍진영' '자베즈' '김우리' '김예원' '휴게소' '이찬혁' '홍윤화' '코팅' '빅맥' '계란'
 '유타' '에일리' '사무라이' '스미스' '유채' '가망' '간이식' '이국주' '임형준' '태용' '와이파이' '혜정' '밀러'
 '핵융합' '손희주' '정경호' '발사체' '몬스터' '요르단' '후보' '빙그레' '엠피' '수자원' '식물' '특보' '추자현'
 '메이커' '감독' '표준' '하바나' '선풍기' '미니스톱' '유빈' '오존' '세로' '도끼' '광해군' '도티' '박지헌'
 '금리' '운트' '한승연' '니콘' '이시언' '탈리스' '김응수' '아르고' '라머' '루커스' '성종' '예원' '이하늬'
 '임다영' '발렌시아' '관사' '백수장' '조성모' '바스프' '정해인' '그래핀' '고유진' '저작물' '안효섭' '휘트니'
 '김경화' '치매' '벤틀리' '써니' '경마' '여주' '식품' '이병헌' '플뢰르' '아플라톡신' '신현준' '그라프'
 '유아인' '냉방' '소희' '청국장' '어코드' '임원희' '김해인' '이필모' '상담' '전통주' '김혜은' '박하나' '웬디'
 '라쿠텐' '감리' '바른' '바닷가재' '렌터카' '레이팅' '김재경' '애견' '혈압계' '위조지폐' '어구' '김종국'
 '플라스틱' '김지우' '유니' '레오' '슈퍼컴퓨터' '무라세' '궤적' '서동주' '특례시' '윤수현' '김성령' '서인영'
 '허영란' '세이퍼트' '화승' '대현' '수영' '순토' '마살라' '박화영' '강경준' '손뜨개' '아우디' '히키' '손성훈'
 '안재욱' '오윤아' '베송' '현아' '서우진' '현서' '고현정' '김경호' '모터쇼' '김가연' '이준기' '양정원'
 '감정평가' '채린' '프린터' '유호정' '최시원' '곰돌이' '연체' '김희철' '쿤타' '양희경' '악사' '동현' '리버풀'
 '크리스탈' '보이차' '안재현

# 이 값들을 이용해 json 데이터를 만들어서 비교하자

In [22]:
print(grid.best_estimator_)

Pipeline(memory=None,
     steps=[('tfidfvectorizer', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=5,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_i...ty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False))])


In [23]:
print(grid.best_estimator_.named_steps["logisticregression"])

LogisticRegression(C=10, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)
